# Air Quality Level Prediction for Pakistani Cities

This notebook demonstrates the complete pipeline for predicting air quality levels in Pakistani cities using machine learning.

## Objectives
- Predict daily AQI category (Good, Moderate, Unhealthy, etc.)
- Forecast 3 days ahead
- Handle time-series data with environmental features

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Add src to path
import sys
sys.path.append('../src')

print("Libraries imported successfully!")

## 1. Data Processing and Cleaning

In [ ]:
# Import data processing module
from data_processing import AirQualityDataProcessor

# Initialize processor
processor = AirQualityDataProcessor()

# Create and process data
print("Creating sample air quality data...")
df_raw = processor.create_sample_data()

print("\nRaw data shape:", df_raw.shape)
print("\nFirst few rows:")
df_raw.head()

In [ ]:
# Clean the data
df_clean = processor.clean_data(df_raw)

print("Cleaned data shape:", df_clean.shape)
print("\nData info:")
df_clean.info()

## 2. Exploratory Data Analysis (EDA)

In [ ]:
# Basic statistics
print("Dataset Summary:")
print(f"Date range: {df_clean['Date'].min()} to {df_clean['Date'].max()}")
print(f"Cities: {list(df_clean['City'].unique())}")
print(f"Total records: {len(df_clean)}")

print("\nAQI Statistics by City:")
df_clean.groupby('City')['AQI'].agg(['mean', 'std', 'min', 'max']).round(2)

In [ ]:
# AQI Category Distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Overall distribution
df_clean['AQI_Category'].value_counts().plot(kind='bar', ax=axes[0])
axes[0].set_title('Overall AQI Category Distribution')
axes[0].set_xlabel('AQI Category')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=45)

# By city
city_aqi = pd.crosstab(df_clean['City'], df_clean['AQI_Category'])
city_aqi.plot(kind='bar', stacked=True, ax=axes[1])
axes[1].set_title('AQI Category Distribution by City')
axes[1].set_xlabel('City')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=45)
axes[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:
# Time series analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# AQI trends by city
for city in df_clean['City'].unique():
    city_data = df_clean[df_clean['City'] == city]
    monthly_avg = city_data.groupby(city_data['Date'].dt.to_period('M'))['AQI'].mean()
    axes[0, 0].plot(monthly_avg.index.astype(str), monthly_avg.values, marker='o', label=city)

axes[0, 0].set_title('Monthly Average AQI Trends by City')
axes[0, 0].set_xlabel('Month')
axes[0, 0].set_ylabel('Average AQI')
axes[0, 0].legend()
axes[0, 0].tick_params(axis='x', rotation=45)

# Seasonal patterns
seasonal_aqi = df_clean.groupby('Month')['AQI'].mean()
axes[0, 1].plot(seasonal_aqi.index, seasonal_aqi.values, marker='o', color='red')
axes[0, 1].set_title('Seasonal AQI Pattern (Average by Month)')
axes[0, 1].set_xlabel('Month')
axes[0, 1].set_ylabel('Average AQI')
axes[0, 1].grid(True)

# Weekly patterns
weekly_aqi = df_clean.groupby('DayOfWeek')['AQI'].mean()
day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
axes[1, 0].bar(day_names, weekly_aqi.values, color='skyblue')
axes[1, 0].set_title('Weekly AQI Pattern (Average by Day of Week)')
axes[1, 0].set_xlabel('Day of Week')
axes[1, 0].set_ylabel('Average AQI')

# Correlation heatmap
numeric_cols = ['AQI', 'PM2.5', 'PM10', 'O3', 'NO2', 'SO2', 'CO', 'Temperature', 'Humidity', 'Wind_Speed', 'Pressure']
corr_matrix = df_clean[numeric_cols].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, ax=axes[1, 1])
axes[1, 1].set_title('Correlation Matrix')

plt.tight_layout()
plt.show()

In [ ]:
# Weather vs AQI analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Temperature vs AQI
axes[0, 0].scatter(df_clean['Temperature'], df_clean['AQI'], alpha=0.5)
axes[0, 0].set_xlabel('Temperature (°C)')
axes[0, 0].set_ylabel('AQI')
axes[0, 0].set_title('Temperature vs AQI')

# Humidity vs AQI
axes[0, 1].scatter(df_clean['Humidity'], df_clean['AQI'], alpha=0.5, color='orange')
axes[0, 1].set_xlabel('Humidity (%)')
axes[0, 1].set_ylabel('AQI')
axes[0, 1].set_title('Humidity vs AQI')

# Wind Speed vs AQI
axes[1, 0].scatter(df_clean['Wind_Speed'], df_clean['AQI'], alpha=0.5, color='green')
axes[1, 0].set_xlabel('Wind Speed (km/h)')
axes[1, 0].set_ylabel('AQI')
axes[1, 0].set_title('Wind Speed vs AQI')

# Pressure vs AQI
axes[1, 1].scatter(df_clean['Pressure'], df_clean['AQI'], alpha=0.5, color='red')
axes[1, 1].set_xlabel('Pressure (hPa)')
axes[1, 1].set_ylabel('AQI')
axes[1, 1].set_title('Pressure vs AQI')

plt.tight_layout()
plt.show()

## 3. Feature Engineering

In [ ]:
# Import feature engineering module
from feature_engineering import FeatureEngineer

# Initialize feature engineer
fe = FeatureEngineer()

# Create features
print("Creating features...")
df_features = fe.prepare_features(df_clean)

print(f"\nOriginal features: {df_clean.shape[1]}")
print(f"Engineered features: {df_features.shape[1]}")
print(f"New features created: {df_features.shape[1] - df_clean.shape[1]}")

In [ ]:
# Show feature types
feature_types = {
    'Lag Features': [col for col in df_features.columns if 'lag' in col],
    'Rolling Features': [col for col in df_features.columns if 'roll' in col],
    'Seasonal Features': [col for col in df_features.columns if any(x in col for x in ['sin', 'cos', 'Season', 'Weekend'])],
    'Interaction Features': [col for col in df_features.columns if any(x in col for x in ['Temp_Humidity', 'Wind_Pressure', 'PM_Ratio'])],
    'Target Features': [col for col in df_features.columns if 'target' in col]
}

for feature_type, features in feature_types.items():
    print(f"\n{feature_type} ({len(features)}):")
    for feature in features[:5]:  # Show first 5
        print(f"  - {feature}")
    if len(features) > 5:
        print(f"  ... and {len(features) - 5} more")

## 4. Model Training and Evaluation

In [ ]:
# Prepare data for modeling
X_train, X_test, y_train, y_test, test_df = fe.get_model_data(df_features)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")
print(f"Target distribution in training set:")
print(y_train.value_counts())

In [ ]:
# Import model training module
from model_training import AQIPredictor

# Initialize predictor
predictor = AQIPredictor()
predictor.feature_engineer = fe

# Train models
print("Training Random Forest...")
rf_model, rf_predictions = predictor.train_random_forest(X_train, y_train, X_test, y_test)

print("\nTraining XGBoost...")
xgb_model, xgb_predictions = predictor.train_xgboost(X_train, y_train, X_test, y_test)

In [ ]:
# Model comparison
print("Model Performance Comparison:")
print("=" * 50)

for model_name, performance in predictor.model_performance.items():
    print(f"{model_name:15} - Accuracy: {performance['accuracy']:.4f}, F1-Score: {performance['f1_score']:.4f}")

# Select best model
best_model, best_model_name = predictor.select_best_model()
print(f"\nBest Model: {best_model_name}")

In [ ]:
# Feature importance analysis
if hasattr(best_model, 'feature_importances_'):
    importance = best_model.feature_importances_
    feature_importance_df = pd.DataFrame({
        'feature': fe.feature_columns,
        'importance': importance
    }).sort_values('importance', ascending=False)
    
    # Plot top 20 features
    plt.figure(figsize=(12, 8))
    top_features = feature_importance_df.head(20)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Feature Importance')
    plt.title(f'{best_model_name} - Top 20 Feature Importance')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    print("\nTop 10 Most Important Features:")
    for i, (_, row) in enumerate(feature_importance_df.head(10).iterrows(), 1):
        print(f"{i:2d}. {row['feature']:30} {row['importance']:.4f}")

## 5. Prediction and Forecasting

In [ ]:
# Save the best model
model_file = predictor.save_model(best_model, best_model_name, fe)
print(f"Model saved to: {model_file}")

# Create prediction outputs
best_predictions = predictor.model_performance[best_model_name]['predictions']
prediction_output = predictor.create_predictions_output(test_df, best_predictions, best_model_name)

print(f"\nPrediction output shape: {prediction_output.shape}")
print("\nSample predictions:")
prediction_output.head(10)

In [ ]:
# Forecast visualization
from prediction import AQIForecastor

# Load the saved model
forecaster = AQIForecastor(model_file)

# Generate forecasts for all cities
cities = df_clean['City'].unique()
latest_date = df_clean['Date'].max()

print(f"Generating 3-day forecasts from {latest_date.date()}...")

all_forecasts = []
for city in cities:
    try:
        forecasts = forecaster.forecast_multiple_days(df_clean, city, latest_date, days=3)
        all_forecasts.extend(forecasts)
        
        print(f"\n{city} Forecast:")
        for forecast in forecasts:
            print(f"  {forecast['Date'].date()}: {forecast['Predicted_AQI_Category']} (Confidence: {forecast['Confidence']:.2f})")
    except Exception as e:
        print(f"Error forecasting for {city}: {e}")

In [ ]:
# Generate alerts
alerts = forecaster.generate_alerts(all_forecasts)

print(f"\nAIR QUALITY ALERTS ({len(alerts)} alerts):")
print("=" * 50)

if alerts:
    for alert in alerts:
        print(f"🚨 {alert['Alert_Level']} ALERT: {alert['City']} - {alert['Date'].date()}")
        print(f"   Category: {alert['Predicted_Category']} (Confidence: {alert['Confidence']:.2f})")
        print(f"   {alert['Message']}")
        print()
else:
    print("✅ No air quality alerts for the forecast period.")

In [ ]:
# City risk ranking
risk_ranking = forecaster.get_city_risk_ranking(df_clean, cities, latest_date)

print(f"\nCITY RISK RANKING for {latest_date.date()}:")
print("=" * 50)

for i, city_risk in enumerate(risk_ranking, 1):
    risk_level = "🔴 HIGH" if city_risk['Risk_Score'] > 3 else "🟡 MEDIUM" if city_risk['Risk_Score'] > 2 else "🟢 LOW"
    print(f"{i:2d}. {city_risk['City']:12} - {city_risk['Predicted_Category']:25} {risk_level} (Score: {city_risk['Risk_Score']:.2f})")

## 6. Model Evaluation Summary

In [ ]:
# Final summary
print("=" * 80)
print("AIR QUALITY PREDICTION MODEL - FINAL SUMMARY")
print("=" * 80)

print(f"\n📊 DATASET INFORMATION:")
print(f"   • Cities: {', '.join(cities)}")
print(f"   • Date Range: {df_clean['Date'].min().date()} to {df_clean['Date'].max().date()}")
print(f"   • Total Records: {len(df_clean):,}")
print(f"   • Features Created: {len(fe.feature_columns)}")

print(f"\n🤖 MODEL PERFORMANCE:")
for model_name, performance in predictor.model_performance.items():
    print(f"   • {model_name:15}: Accuracy = {performance['accuracy']:.3f}, F1-Score = {performance['f1_score']:.3f}")

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   • Accuracy: {predictor.model_performance[best_model_name]['accuracy']:.3f}")
print(f"   • F1-Score: {predictor.model_performance[best_model_name]['f1_score']:.3f}")

print(f"\n📁 OUTPUT FILES:")
print(f"   • Model: {model_file}")
print(f"   • Predictions: outputs/aqi_predictions_{best_model_name}.csv")
print(f"   • Confusion Matrix: outputs/{best_model_name.lower()}_confusion_matrix.png")
print(f"   • Feature Importance: outputs/{best_model_name.lower()}_feature_importance.png")

print(f"\n⚠️  ALERTS GENERATED: {len(alerts)}")
if alerts:
    alert_cities = list(set([alert['City'] for alert in alerts]))
    print(f"   • Cities with alerts: {', '.join(alert_cities)}")

print(f"\n🚀 NEXT STEPS:")
print(f"   • Run Streamlit app: streamlit run app.py")
print(f"   • View interactive dashboard with forecasts")
print(f"   • Monitor air quality alerts for Pakistani cities")

print("\n" + "=" * 80)